In [2]:
import os 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from  IPython.display import display
import plotly.express as px

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, experimental, MaxPool2D, BatchNormalization
from tensorflow.keras.losses import sparse_categorical_crossentropy, binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau 
from tensorflow.data import Dataset
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.random import set_seed
from tensorflow.keras.utils import to_categorical
from tensorflow import test
import random


In [ ]:
#link to the dataset: https://www.kaggle.com/jatingoyal123/age-and-gender-classifier/?select=age_gender.csv 

In [3]:
#setting the seeds-
np.random.seed(11)
set_seed(11)
random.seed(11)

In [11]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#!wget age_gender.csv       
#age_gender_data = pd.read_csv("/kaggle/input/age-gender-and-ethnicity-face-data-csv/age_gender.csv")
#age_gender_data.info()
!curl -O https://www.kaggle.com/jatingoyal123/age-and-gender-classifier/?select=age_gender.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:19 --:--:--     0curl: (6) Could not resolve host: www.kaggle.com


In [ ]:
age_gender_data.head()

In [ ]:
#plot age
sns.countplot(x='age', data=age_gender_data

In [ ]:
#plot gender
sns.countplot(x='gender', data=age_gender_data)

In [ ]:
#preprocessing chart
age_gender_data = age_gender_data[age_gender_data['age'] >= 18]
sns.countplot(x='age', data=age_gender_data)

In [ ]:
#preprocessing dataset table
age_gender_data.reset_index(drop=True, inplace=True)
age_gender_data

In [ ]:
#checking for null data
age_gender_data.isnull().sum()

In [ ]:
# sixe of images explained
num_pixels = len(age_gender_data['pixels'][0].split(' '))
dimension = int(np.sqrt(num_pixels))
img_width = dimension
img_height = dimension

print("Pixels: {}".format(num_pixels))
print("Width: {0}, Height: {1}".format(img_width, img_height))

In [ ]:
#splitting the data

X_img = age_gender_data.iloc[:,4].copy()

y_age=age_gender_data.iloc[:,0].copy()

y_ethnicity = age_gender_data.iloc[:,1].copy()

y_gender = age_gender_data.iloc[:,2].copy()

X_train, t_te, y_train, y_te = train_test_split(X_img, y_gender, test_size=0.3, random_state=11)

X_val, X_test, y_val, y_test = train_test_split(X_te, y_te, test_size=0.15, random_stat=11)

def str_to_npArr(x):
    x = x.reset_index(drop=True)
    
    x = x.apply(lambda x: np.array(x.split(), dtype="float32")) 
    
    return np.array([x[i].reshape(img_width, img_height, 1) for i in range(x.shape[0])])


X_train = str_to_npArr(X_train)

X_test = str_to_npArr(X_test)

X_val = str_to_npArr(X_val)

print("Traget: shape = (16593, 48, 48, 1), type = <class 'numpy.ndarray'>")

print("Current: shape = {}, type = {}".format(X_train.shape, type(X_train)))

In [ ]:
target_colums = ['gender', 'ethnicity', 'age']

age_gender_data_preprocess = age_gender_data.drop('img_name', axis=1)

y = age_gender_data_preprocess[target_columns]

X = age_gender_data_preprocess.drop(target_columns, axis=1)

print(X)
print("------------------------------------------------------------------------")
print(y)

In [ ]:
X = X['pixels'].apply(lambda c: np.array(x.split(), dtype="float32"))

X = np.array(X)/255.0

X = np.array([ X[i].reshape(48,48,1) for i in range (X.shape[0])])

print("Traget: X Shape: {}".format(X.shape))
print("Current: X Shape: {}".format(X.shape))

In [ ]:
y_gender = np.array(y['gender'])

y_ethnicity = np.array(y['ethnicity'])

y_age = np.array(y['age'])

In [ ]:
rows = 20 
cols= 5
samp=random.sample(range(X.shape[0]), rows*cols)
    
x_samp = X[samp,:,:,:]
y_samp_gender = y_gender[samp]
y_samp_age = y_age[samp]

fig,ax=plt.subplots(rows, cols, figsize=(16,60))
r=0
c=0

for i in range(rows*cols):
    aa = x_samp[i,:,:,:].reshape(48,48)
    ax[r,c].axis("off")
    ax[r,c].imshow(aa,cmap="gray")
    
    ax[r,c].set_title(f"Gender: {'Female' if y_samp_gender[i]==1 else 'Male'}, Age: {y_samp_age[i]}")
    
    c+=1
    if c == cols:
        c=0
        r+=1
        
plt.show()

In [ ]:
train_data_gen = ImageDataGenerator(rotation_range=30, width_shift_range=1, brightness_range=[0.8,1.2],zoom_range=[0.8,1.2],rescale=1/255)

val_data_gen = ImageDataGenerator(rescale=1/255)

test_data_gen = ImageDataGenerator(rescale=1/255)

In [ ]:
set_seed(11)
random.seed(11)
np.random.seed(11)

val_data = val_data_gen.flow(X_val,y_val, seed=11,shuffle=False)

tst_data = test_data_gen.flow(X_test,y_test, seed=11,shuffle=False)

In [ ]:
fig, ax = plt.subplots(10, 5, figsize=(15,25))
for n in range(10):
    r = random.sample(range(X.shape[0]), 1)[0]
    ax[n,0].imshow(X[r].reshape(48,48),cmap="gray")
    ax[n,0].set_title("Original")
    ax[n,0].axis("off")
    for i in range(1,5):
        ax[n,i].imshow(train_data_gen.random_transform(X[r]).reshape(48,48),cmap="gray")
        ax[n,i].set_title("Augmented")
        ax[n,i].axis("off")
plt.show()

In [ ]:
#training

batch_size = 32
imge_width, img_height, img_num_channels = 48, 48, 1
loss_function = sparse_categorical_crossentropy
no_classes=2
no_epochs=50
optimizer=Adam()
verbosity=1
num_folds=10
activation='softmax'

input_shape = (img_width, img_height, img_num_channels)
input_shape

In [ ]:
random.seed(11)
set_seed(11)
np.random.seed(11)

kfold = KFold(n_splits=num_folds, shuffle=True)

In [ ]:
acc_per_fold = []
loss_per_fold = []

fold_no=1
for train test kfold.split(X, y_gender):
    random.seed(11)
    set_seed(11)
    np.random.seed(11)

In [ ]:
model = Sequential()
  
model.add(Conv2D(64, kernel_size=(3,3), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Conv2D(512, kernel_size=(3,3), activation='relu', padding='same'))model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='softmax'))


In [ ]:
model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=['accuracy'])
  
print("------------------------------------------------------------------------------------------")
print(f"Training for fold {fold_no} ...")
    
early_stop = EarlyStopping(monitor="val_loss",patience=5,mode="min") 

In [ ]:
random.seed(11)
set_seed(11)
np.random.seed(11)
    
history = model.fit(train_data_gen.flow(X[train], y_gender[train], seed=11), callbacks=early_stop, batch_size=batch_size, epochs=no_epochs, verbose=verbosity,
        validation_data=train_data_gen.flow(X[test], y_gender[test],seed=11))
  

In [ ]:
fig = px.line(history.history, y=['loss', 'val_loss'], labels={'index': 'epoch', 'value': 'loss'}, 
              title='Training History')
fig.show()
    
scores = model.evaluate(train_data_gen.flow(X[test], y_gender[test], seed=11), verbose=0)
print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; 
      {model.metrics_names[1]} of {scores[1]*100}%')
acc_per_fold.append(scores[1] * 100)
loss_per_fold.append(scores[0])
fold_no = fold_no + 1

In [ ]:
#average scores

print("------------------------------------------------------------------------------------------")
print("Score per fold")
for i in range(0, len(acc_per_fold)):
    print("------------------------------------------------------------------------------------------")
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print("------------------------------------------------------------------------------------------")
print("Average scores for all folds:")
print(f"> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})")
print(f"> Loss: {np.mean(loss_per_fold)}")
print("------------------------------------------------------------------------------------------")

In [ ]:
model = Sequential()
  
model.add(Conv2D(64, kernel_size=(3,3), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Conv2D(512, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='softmax'))

model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
Final_train = np.append(X_train, X_val, axis=0)
Final_val = np.append(y_train, y_val, axis=0)
final_training_data = train_data_gen.flow(Final_train, Final_val,seed=11)

random.seed(11)
set_seed(11)
np.random.seed(11)
final_model_history = model.fit(train_data_gen.flow(X, y_gender, seed=11),batch_size=32,epochs=20, validation_data=val_data)

In [ ]:
fig = px.line(
final_model_history.hisotry, y=["loss", "val_loss"],
labels={"index": "epoch", "value": "val_loss"}, 
title='Training History')
fig.show()

fig = px.line(
final_model_history.hisotry, y=["loss", "val_loss"],
labels={"index": "epoch", "value": "val_loss"}, 
title='Training History')

fig.show()

In [ ]:
pwd

In [ ]:
model.save("backup")

In [ ]:
model.evaluate(test_data)

In [ ]:
y_pred = model.predict_classes(test_data)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
cm = confuation_matrix(y_test, y_pred)
sns.heatmap(cm, cmap="Greens", cbar=False, annot+true, fmt="d");

In [ ]:
error_index = (y_test != y_pred)
y_test_error = y_test[error_index]
X_test_error = X_test[error_index]
prediction_error = y_pred[error_index]

In [ ]:
rows=int(np.floor(sum(error_index)/3)) 
cols=3 
x_samp = X_test_error
y_samp = y_test_error

fig,ax = plt.subplots(rows,cols,figsize=(15,50))
r = 0
c = 0
for i in range((rows*cols)-1):
    aa = x_samp[i].reshape(48,48)
    ax[r,c].axis("off")
    ax[r,c].imshow(aa,cmap="gray")
    actual_lab = "Female" if y_samp.iloc[i]==1 else "Male"
    pred_lab = "Female" if int(prediction_error[i])==1 else "Male"
    ax[r,c].set_title(f'Actual: {actual_lab}\nPred: {pred_lab}')
    c+=1
    if c == cols:
        c=0
        r+=1
plt.show()

In [ ]:
import cv2

In [ ]:
image =cv2.imread('../input/testset/mind-long.jpg',0)
plt.imshow(img, cmap="gray")
img = cv2.resize(img, (48, 48))
img = np.reshape(img,[1,48,48,1])
img_pixels = img.astype("float32") / 255.0
classes = model.predict_classes(img_pixels)

mapper=['male','female']
print(mapper[classes[0]])